In [ ]:
import numpy as np
import pandas as pd

In [ ]:
eval_metadata = pd.read_csv('evaluation_data/GSE159437_filtered_metadata.csv')
eval_counts = pd.read_csv('evaluation_data/GSE159437_GeneLevel_Raw_data.csv')

In [ ]:
eval_counts.head()

In [ ]:
eval_metadata.head()

In [ ]:
eval_metadata['disease state'].value_counts()

In [ ]:
selected_gene_df = pd.read_csv('results/pe_sle_hpt_hpth_multimode_selected_features.csv')
selected_gene_df.head()

In [ ]:
eval_counts = eval_counts.drop(columns=['Unnamed: 0'])
eval_data = eval_counts.set_index('gene_symbol').T
eval_data.index.name = "GSM"
eval_data['label'] = eval_metadata['disease state'].values
eval_data.columns = [col if isinstance(col, str) else col[0] for col in eval_data.columns]

In [ ]:
selected_genes = selected_gene_df['Feature']
selected_genes = sorted(selected_genes)
eval_data = eval_data[selected_genes + ['label']]
eval_data.head()

In [ ]:
duplicates = eval_data.columns.duplicated()
eval_data = eval_data.loc[:, ~duplicates]
eval_data.to_csv("clean_data/eval_data.csv", index=False)

In [ ]:
X = eval_data.drop("label", axis=1)
y = eval_data["label"]

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)
print("Scaled features shape:", X.shape)

y = y.replace({"Normal": 0, "Severe Preeclampsia": 1})

In [ ]:
import joblib

model = joblib.load("models/best_Gradient Boosting.pkl")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred, average="macro", zero_division=0)
recall = recall_score(y, y_pred, average="macro", zero_division=0)
f1 = f1_score(y, y_pred, average="macro", zero_division=0)
roc_auc = roc_auc_score(y, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)
print("ROC AUC:", roc_auc)